# CPDCTL samples for Notebooks and Environment in Projects

<span style="color:red">**Note: This notebooks is the [sample notebook](https://github.com/IBM/cpdctl/blob/master/samples/Notebook-and-Environment-samples-for-Projects.ipynb) from the cpdctl public repo. And tested and updated in CPD 4.0.4 - JupyterLab environment for latest cpdctl releases 1.1.132 by WW Data and AI team.**</span>


CPDCTL is a command-line interface (CLI) you can use to manage the lifecycle of notebooks. By using the notebook CLI, you can automate the flow for creating notebooks and running notebook jobs, moving notebooks between projects in Watson Studio, and adding custom libraries to notebook runtime environments.

This notebook begins by showing you how to install and configure CPDCTL and is then split up into four sections with examples of how to use the commands for:

- Creating notebooks and running notebook jobs
- Creating Python scripts and running script jobs
- Downloading notebooks from one project and uploading them to another project
- Adding custom libraries to a notebook runtime environment

## 0. Before you begin - setup

### 0.1 Import required libraries and modules

In [6]:
# Import required libraries and modules
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import jmespath
import subprocess
from IPython.core.display import display, HTML

### 0.2 Install the version v1.1.132 of `cpdctl`

In [7]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')
version='v1.1.132'

response = requests.get(CPDCTL_RELEASES_URL)
asset_version = next(a for a in response.json() if version==a['tag_name'])
#assets = response.json()[0]['assets']
assets=asset_version['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [8]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [9]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.132


In [10]:
!which cpdctl

/userfs/Notebooks/cpdctl


In [11]:
!cpdctl version

1.1.132


## 1. Provide CPD Cluster credentials

In [12]:
CPD_USERNAME = 'admin' # for example: datascientist
CPD_PASSWORD = 'CP4DDataFabric'
CPD_URL = 'https://datafabric.ibmcloudpack.com:8954' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

Since this notebook runs inside of the CPD cluster you want to interact with the cpdctl tool, you can leverage the zeror configration mode, which automatcially connects to the CP4D instance.

In [13]:
# show all project
! cpdctl project list

...
ID                                     Name              Created                    Description   Tags   
035869f9-b021-455a-980a-3ed56e3d722f   New Git           2022-03-30T19:32:12.873Z                 []   
f8b97b79-18f8-49f4-9351-99dbf62c4c9a   Classic Project   2022-03-30T18:49:42.321Z                 []   


In [14]:
# show all deployment spaces
! cpdctl space list

...
ID                                     Name         Created                    Description   State    Tags   
d9737768-f15b-4f36-ab58-007c58558986   Demo Space   2022-03-30T18:57:08.544Z                 active   []   


## 2. Choose a project you want to work with

<span style="color:red">Important Note: If you want to interact with classic notebooks, you have to choose a **classic project that is not configured with git integration** OR a deployment space.</span>

In [15]:
# You can specify your project id directly:
project_id = "f8b97b79-18f8-49f4-9351-99dbf62c4c9a"

# OR you can reference it by index[0] is the index of the project id list
#result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
#project_id = result.s

print("project id: {}".format(project_id))

project id: f8b97b79-18f8-49f4-9351-99dbf62c4c9a


## 3. Create a notebook in a non-git project using cpdctl

First of all, you need to create a notebook asset in your project. To create a notebook asset you need to specify:

- The environment in which your notebook is to run
- A notebook file (.ipynb).

### 3.1 Get the environment ID

List all the environments in your project, filter them by their display name and get the ID of the environment in which your notebook will be run:

In [54]:
# You can find the name of the available environments in the project => Environment tab

environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [55]:
result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-f8b97b79-18f8-49f4-9351-99dbf62c4c9a


### 3.2 Upload the .ipynb file

The notebook I want to upload is called *Read_Write_CSV_Data.ipynb*, and it is in my current working directory. And I want to upload it to the target project.

In [57]:
remote_file_path = "notebook/Read_Write_CSV_Data.ipynb"
local_file_path = "Read_Write_CSV_Data.ipynb"

In [58]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


### 3.3 Create a notebook asset: associate environment runtime with the notebook file

In [59]:
file_name = "Read_Write_CSV_Data.ipynb"

runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)

originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

In [60]:
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project {project_id} --runtime '{runtime_json}' --originates-from '{originate_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: 42443869-64db-49ab-a039-f96d5b042623


<span style="color:red">Important Note: Now you can check the target project (eg.in my case, Classic Project), the Read_Write_CSV_Data notebook should show up in the Notebook section of the Assets tab. </span>

## 4. Create and run a notebook job

In [61]:
result = ! cpdctl notebook version create --notebook-id {notebook_id} --output json -j "metadata.guid" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

version id: e2d9f014-c121-44ab-88ff-c03f0c52cb2b


In [24]:
! cpdctl notebook version create --notebook-id {notebook_id} --output json -j "metadata.guid" --raw-output

ef98f57d-d81d-49f6-9b64-aac963970e23


In [32]:
job_name = "cpdctl-test-job"
job = {
    'asset_ref': notebook_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
           # 'foo=1', 
           # 'bar=2'
        ]
    }, 
    'description': 'my job', 
    'name': job_name
}
job_json = json.dumps(job)

In [33]:
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: 094ba812-0611-44f0-95ac-86c2eb1cd448


In [34]:
job_run = {
    'configuration': {
        'env_variables': [
            #'key1=value1', 
            #'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

In [35]:
result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: c8dc6dea-addd-4c8f-867f-20a6f10e158c


In [36]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

Cell 4:

Cell 8:

Cell 11:




## 5. Creating a Python script asset and running a job

<span style="color:red">Before starting with this section, ensure that you have **cpdctl v1.1.132 installed (see Section 0)** and specified the **CP4D Cluster** and **ID of the project in which you will work (see Section 1 and 2)**.</span>

Suppose you have a Python script (.py) on your local system and you would like to run the code in the script as a job on a CPD cluster. This section shows you how to create a Python script asset and run a job on a CPD cluster.

### 5.1 Creating a Python script asset

In [128]:
# upload the scipt .py file
remote_file_path = "script/test_script.py"
local_file_path = "code_pkg/hello.py"

In [129]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


In [130]:
# Specify the metadata, entity and attachments of the script file:

metadata = {
    "name": "my_test_script_with_env_var",
    "asset_type": "script",
    "asset_category": "USER",
    "origin_country": "us"
}
metadata_json = json.dumps(metadata)

entity = {
    "script": {
        "language": {
            "name": "python3"
        }
    }
}
entity_json = json.dumps(entity)

attachments = [
    {
        "asset_type": "script",
        "name": "my_test_script",
        "description": "attachment for script",
        "mime": "application/text",
        "object_key": remote_file_path
    }
]
attachments_json = json.dumps(attachments)

In [131]:
# Create a Python script asset:

result = ! cpdctl asset create  --metadata '{metadata_json}' --entity '{entity_json}' --attachments '{attachments_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
script_id = result.s
print("script id: {}".format(script_id))

script id: 3e14540b-3092-47d8-9942-48eb8289a8c8


Check the target project now, you should see the script under the Asset tab.

### 5.2 Running a script job

In [132]:
# specify environment by id
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-f8b97b79-18f8-49f4-9351-99dbf62c4c9a


Now you can create a script job. To do this, you need to give your script job a name, a description, and pass the script ID and environment ID.

In [133]:
# define job metadatat
job_name = "cpdctl-test-job-for-script_with_env_v3"
job = {
    'asset_ref': script_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'input=hello', 
            #'bar=2'
        ]
    }, 
    'description': 'my script job', 
    'name': job_name
}
job_json = json.dumps(job)

# create job
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: 1d7b06f4-358b-4309-b4c1-32e5e23fa628


In [134]:
# Kick off the job
job_run = {
    'configuration': {
        'env_variables': [
            'input=hello', 
            #'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: 6868f64a-efa3-4d6d-abbf-103357f51765


In [135]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

hello




## 6. Downloading a notebook and uploading it to another project

<span style="color:red">Before starting with this section, ensure that you have **cpdctl v1.1.132 installed (see Section 0)** and specified the **CP4D Cluster** and **ID of the project in which you will work (see Section 1 and 2)**.</span>


Suppose you have a notebook in one project and would like to add a specific version of this notebook to another project. To do this, you first need to download the notebook file to your local system and then upload it to the other project. After that you need to create a notebook asset in your project by referencing the uploaded notebook file (.ipynb) and specifying the environment in which your notebook is to run.

### 6.1 Download a notebook

In [22]:
notebook_id = "42443869-64db-49ab-a039-f96d5b042623"

In [23]:
# list notebook versions
! cpdctl notebook version list --notebook-id {notebook_id}

...
ID                                     Created   
e2d9f014-c121-44ab-88ff-c03f0c52cb2b   1649188298833   


### 6.2 Get the path in the storage volume to the notebook version that you want to download:

In [26]:
result = ! cpdctl notebook version list --notebook-id {notebook_id} --output json -j "(resources[].metadata.guid)[0]" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

# You can also specify your version id directly:
# version_id = "Your version ID"

version id: e2d9f014-c121-44ab-88ff-c03f0c52cb2b


In [27]:
result = ! cpdctl notebook version get --notebook-id {notebook_id} --version-id {version_id} --output json -j "entity.file_reference" --raw-output
version_storage_path = result.s
print("version storage path: {}".format(version_storage_path))

version storage path: notebook/attachment_for_notebook_42443869_64db_49ab_a039_f96d5b042623_6da56e36xeazd6ltp5hasd3af.ipynb


### 6.3 Download the noteboook asset with the specific version from the storage path:

In [28]:
file_name = "my-new-notebook.ipynb"

! cpdctl asset file download --path {version_storage_path} --output-file {file_name} --project-id {project_id} --raw-output

...
OK
Output written to my-new-notebook.ipynb


### 6.4 Upload the notebook to another project

In [32]:
# if you dont have another project, you can create one using line below
#!cpdctl project create --name "new classic project"

...
              
location   /v2/projects/d7d9f3e3-db92-4248-83a7-d4f7835dcec3   


In [33]:
! cpdctl project list

...
ID                                     Name                  Created                    Description   Tags   
035869f9-b021-455a-980a-3ed56e3d722f   New Git               2022-03-30T19:32:12.873Z                 []   
d7d9f3e3-db92-4248-83a7-d4f7835dcec3   new classic project   2022-04-06T15:44:32.180Z                 []   
f8b97b79-18f8-49f4-9351-99dbf62c4c9a   Classic Project       2022-03-30T18:49:42.321Z                 []   


In [34]:
# pick project by index, in this case, index 1 refers to the new classic project we just created
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[1]" --raw-output
project2_id = result.s
print("another project id: {}".format(project2_id))

# You can also specify your another project id directly:
# project2_id = "Your another project ID"

another project id: d7d9f3e3-db92-4248-83a7-d4f7835dcec3


In [36]:
# Upload the notebook file to this project:
remote_file_path = "notebook/{}".format(file_name)

! cpdctl asset file upload --path {remote_file_path} --file {file_name} --project-id {project2_id}

...
OK


### 6.5 Specify environment by id

In [38]:
# identify environment id
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

result = ! cpdctl environment list --project-id {project2_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-d7d9f3e3-db92-4248-83a7-d4f7835dcec3


### 6.6 Associate environment runtime with the notebook file 

In [40]:
#Now you can create a notebook asset in this project by referencing the uploaded notebook file and attaching it to the environment runtime:

file_name = "my-new-notebook-in-another-project.ipynb"

runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)

originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project {project2_id} --originates-from '{originate_json}' --runtime '{runtime_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: 0cce3597-2ea3-40df-8769-282f4a4a7d91


Now if you check the new classic project, there should be a notebook named my-new-notebook-in-another-project.ipynb

## 7. Adding additional packages for custom environment

<span style="color:red">Before starting with this section, ensure that you have **cpdctl v1.1.132 installed (see Section 0)** and specified the **CP4D Cluster** and **ID of the project in which you will work (see Section 1 and 2)**.</span>

Suppose you have a **conda-yml** file that lists your additional packages or you have a pip-zip file containing your custom packages, and you would like to install these packages in your custom environment. To do this, you need to:

- Create a custom software specification
- Add your custom packages
- Create a custom environment

### 7.1 Idenfity the base image you want to customize by id

To create a custom software specification, you need to specify the base software specification that you want to customize. You can list all the software specifications in your project and choose one of them as the base software specification:

In [46]:
# double check which project_id you are using
#!cpdctl project list 

...
ID                                     Name                  Created                    Description   Tags   
035869f9-b021-455a-980a-3ed56e3d722f   New Git               2022-03-30T19:32:12.873Z                 []   
d7d9f3e3-db92-4248-83a7-d4f7835dcec3   new classic project   2022-04-06T15:44:32.180Z                 []   
f8b97b79-18f8-49f4-9351-99dbf62c4c9a   Classic Project       2022-03-30T18:49:42.321Z                 []   


In [62]:
project_id

'f8b97b79-18f8-49f4-9351-99dbf62c4c9a'

In [43]:
! cpdctl environment software-specification list --project-id {project_id} 

...
ID                                     Name                            Created                    Description                                          Type   
0062b8c9-8b7d-44a0-a9b9-46c416adcbd9   default_py3.6                   2022-01-05T15:30:14.436Z   Default Python 3.6                                   software_specification   
069ea134-3346-5748-b513-49120e15d288   pytorch-onnx_1.3-py3.7-edt      2022-01-05T15:30:14.485Z   Software specification for Pytorch 1.3.1 Elastic …   software_specification   
09c5a1d0-9c1e-4473-a344-eb7b665ff687   scikit-learn_0.20-py3.6         2022-01-05T15:30:14.477Z   Software specification for Scikit-learn on Python…   software_specification   
09f4cff0-90a7-5899-b9ed-1ef348aebdee   spark-mllib_3.0-scala_2.12      2022-01-05T15:30:14.497Z   Machine Learning on Spark 3.0 with Scala 2.12        software_specification   
0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda   ai-function_0.1-py3.6           2022-01-05T15:30:14.434Z   Software specification for AI f

In [48]:
base_sw_spec_name = "Default Python 3.8"
query_string = "(resources[?metadata.description == '{}'].metadata.asset_id)[0]".format(base_sw_spec_name)

result = ! cpdctl environment software-specification list --project-id {project_id} --output json -j "{query_string}" --raw-output
base_sw_spec_id = result.s
print("base software specification id: {}".format(base_sw_spec_id))

# You can also specify your base software specification id directly:
# based_sw_spec_id = "Your base software specification ID"

base software specification id: ab9e1b80-f2ce-592c-a7d2-4f2344f77194


### 7.2 Create a custom software specification

In [49]:
custom_sw_spec_name = "my_sw_spec"

base_sw_spec = {
    'guid': base_sw_spec_id
}

base_sw_spec_json = json.dumps(base_sw_spec)

sw_conf = {}
sw_conf_json = json.dumps(sw_conf)

In [50]:
result = ! cpdctl environment software-specification create --project-id {project_id} --name {custom_sw_spec_name} --base-software-specification '{base_sw_spec_json}' --software-configuration '{sw_conf_json}' --output json -j "metadata.asset_id" --raw-output
custom_sw_spec_id = result.s
print("custom software specification id: {}".format(custom_sw_spec_id))

custom software specification id: 0a99d940-d0da-4204-8e55-a318f72e5592


### 7.3 Create a package extension and get its path

In [ ]:
pkg_name = "my_test_packages"

In [78]:
#result = ! cpdctl environment package-extension create --name {pkg_name} --type "conda_yml" --project-id {project_id}  --output json
result = ! cpdctl environment package-extension create --name 'my_test_packages' --type "conda_yml" --project-id 'f8b97b79-18f8-49f4-9351-99dbf62c4c9a'  --output json
pkg_ext_id = json.loads(result.s)['metadata']['asset_id']
print("package extension id: {}".format(pkg_ext_id))

package extension id: 4f5ff287-de8f-4633-9f6c-4d982d1ed3d6


Get the path to where you want to upload the additional packages:

In [80]:
pkg_ext_href = json.loads(result.s)['entity']['package_extension']['href'].split('/')[4].split('?')[0]
remote_pkg_path = "package_extension/{}".format(pkg_ext_href)
print("path where asset should be uploaded: {}".format(remote_pkg_path))

path where asset should be uploaded: package_extension/my_test_packages_5DBAcnSUF.yml


### 7.4 Define a conda-yaml file listing additional packages:

In [81]:
my_yaml = """
channels:
  - defaults

dependencies:
  - pip:
    - fuzzywuzzy

"""

with open('my-pkg-ext.yaml', 'w') as f:
    f.write(my_yaml)

### 7.5 Upload the yaml file to the path returned in the previous command:

In [82]:
local_pkg_path = "my-pkg-ext.yaml"

In [83]:
! cpdctl asset file upload --path "{remote_pkg_path}" --file {local_pkg_path} --project-id {project_id} 

...
OK


### 7.6 Confirm the upload of the library file attached to the package extension is complete

In [84]:
! cpdctl environment package-extension upload-complete --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


### 7.7 add package extensions (yml file) to the custom software specification (base image)

In [85]:
! cpdctl environment software-specification add-package-extensions --software-specification-id {custom_sw_spec_id} --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


### 7.8 Configure hardware specification

In [86]:
! cpdctl environment hardware-specification list --project-id {project_id} 

...
ID                                     Name            Created                    Description                                          Type   
5a1f0e64-e420-55ce-bd7a-f6d00bb942cf   ML              2022-01-05T15:30:15.797Z   A hardware specification providing 4 CPU cores an…   hardware_specification   
a02f3ab5-6964-4f06-a870-c7cc69187895   V100x2          2022-01-05T15:30:15.798Z   A hardware specification providing 52 CPU cores a…   hardware_specification   
a6c4923b-b8e4-444c-9f43-8a7ec3020110   L               2022-01-05T15:30:15.796Z   A hardware specification providing 8 CPU cores an…   hardware_specification   
ac59d20a-9c7c-4504-a853-788ef35969da   Default Spark   2022-01-05T15:30:15.794Z   A hardware specification for Spark with 1 CPU and…   hardware_specification   
b128f957-581d-46d0-95b6-8af5cd5be580   XXS             2022-01-05T15:30:15.800Z   A hardware specification providing one CPU core a…   hardware_specification   
b2232f7a-bfad-4822-9bce-6ba1af49217a   M-Spark  

In [87]:
hw_spec_keyword_1 = "one CPU core"
hw_spec_keyword_2 = "4 GiB of memory"
query_string = "(resources[?contains(metadata.description, '{}') && contains(metadata.description, '{}')].metadata.asset_id)[0]".format(hw_spec_keyword_1, hw_spec_keyword_2)

result = ! cpdctl environment hardware-specification list --project-id {project_id}  --output json -j "{query_string}" --raw-output
hw_spec_id = result.s
print("hardware specification id: {}".format(hw_spec_id))

# You can also specify your hardware specification id directly:
# hw_spec_id = "Your base software specification ID"

hardware specification id: f3ebac7d-0a75-410c-8b48-a931428cc4c5


### 7.9 Create a custom environment by specifying the hardware specification, the custom software specification and the tool specification:

In [89]:
env_name = "my_custom_env"

hw_spec = {
    'guid': hw_spec_id
}

custom_sw_spec = {
    'guid': custom_sw_spec_id
}

custom_sw_spec_json = json.dumps(custom_sw_spec)

tool_spec = {
    'supported_kernels': [{
        'language': 'python', 
        'version': '3.8', 
        'display_name': 'Python 3.8'
    }]
}
hw_spec_json = json.dumps(hw_spec)

tool_spec_json = json.dumps(tool_spec)

In [90]:
result = ! cpdctl environment create --project-id {project_id} --type "notebook" --name {env_name} --display-name {env_name} --hardware-specification '{hw_spec_json}' --software-specification '{custom_sw_spec_json}' --tools-specification '{tool_spec_json}' --output json -j "metadata.asset_id" --raw-output
custom_env_id = result.s
print("custom environment id: {}".format(custom_env_id))

custom environment id: 08cf9572-ca33-4134-b745-830d7a21239b


Now you can go to your target project where you just created a custom environment and test if it works properly.